<a href="https://colab.research.google.com/github/sg3451/Colab_Notebooks/blob/main/RAG_1_claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this code is copied from that generated by Claude Sonnet 3.5
#the purpose of this code is to use RAG to query LLMs
#this code chunk is to set up the environment
python -m venv rag_env
source rag_env/bin/activate  # On Windows, use `rag_env\Scripts\activate`
pip install pypdf sentence-transformers faiss-cpu anthropic flask

To use this system:

1. Set up your local directories for PDFs, embeddings, and the FAISS index.
2. Run the rag_pipeline function once to process documents, generate embeddings, and create the index.
3. On subsequent runs, it will load the pre-computed embeddings and index, making queries faster.

In [ ]:
# Document processing :create a script to extract text from my PDF documents
import os
   from pypdf import PdfReader

   def extract_text_from_pdf(pdf_path):
       reader = PdfReader(pdf_path)
       text = ""
       for page in reader.pages:
           text += page.extract_text() + "\n"
       return text

   def process_documents(directory):
       documents = []
       for filename in os.listdir(directory):
           if filename.endswith(".pdf"):
               file_path = os.path.join(directory, filename)
               text = extract_text_from_pdf(file_path)
               documents.append({"filename": filename, "text": text})
       return documents

   # Usage
   pdf_directory = "path/to/your/pdfs" #specify the path to folder where the pdfs are stored
   processed_docs = process_documents(pdf_directory)

In [ ]:
#Text embedding : Create a function to embed the extracted text using sentence-transformers
from sentence_transformers import SentenceTransformer

   def embed_documents(documents, model_name='all-MiniLM-L6-v2'): #model name can be changed
       model = SentenceTransformer(model_name)
       embeddings = []
       for doc in documents:
           embedding = model.encode(doc['text'])
           embeddings.append(embedding)
       return embeddings

   # Usage
   embeddings = embed_documents(processed_docs)

In [ ]:
#The embeddings are generated and stored in memory in the current implementation. To persist them locally, we can add a function to save and load embeddings
import numpy as np
import os

def save_embeddings(embeddings, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i, embedding in enumerate(embeddings):
        np.save(os.path.join(directory, f"embedding_{i}.npy"), embedding)

def load_embeddings(directory):
    embeddings = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".npy"):
            embedding = np.load(os.path.join(directory, filename))
            embeddings.append(embedding)
    return embeddings

# Usage
embeddings_directory = "path/to/local/embeddings"
save_embeddings(embeddings, embeddings_directory)
loaded_embeddings = load_embeddings(embeddings_directory)

In [ ]:
#Vector Database:Use FAISS to create and search a vector database
import faiss
import numpy as np

def create_faiss_index(embeddings):
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))
    return index

def search_similar_documents(index, query_embedding, k=5):
    distances, indices = index.search(np.array([query_embedding]).astype('float32'), k)
    return indices[0]

def save_faiss_index(index, filepath):
    faiss.write_index(index, filepath)

def load_faiss_index(filepath):
    return faiss.read_index(filepath)

# Usage
faiss_index = create_faiss_index(embeddings)
index_filepath = "path/to/local/faiss_index.bin"
save_faiss_index(faiss_index, index_filepath)
loaded_index = load_faiss_index(index_filepath)

In [ ]:
#Integration with Claude:Use the Anthropic API to interact with Claude
from anthropic import Anthropic

   anthropic = Anthropic(api_key="your-api-key-here")

   def query_claude(query, context):
       prompt = f"Context: {context}\n\nQuery: {query}\n\nResponse:"
       response = anthropic.completions.create(
           model="claude-3-sonnet-20240229",
           max_tokens_to_sample=300,
           prompt=prompt
       )
       return response.completion

   # Usage
   response = query_claude("What is the main topic?", "Here's some context...")

In [ ]:
#Orchestration:Create a main script to tie everything together
from document_processor import process_documents
from text_embedder import embed_documents
from vector_database import create_faiss_index, search_similar_documents, save_faiss_index, load_faiss_index
from local_embeddings_handler import save_embeddings, load_embeddings
from claude_integration import query_claude
import os

def rag_pipeline(query, pdf_directory, embeddings_directory, index_filepath):
    # Process documents if not already processed
    if not os.path.exists(embeddings_directory):
        documents = process_documents(pdf_directory)
        embeddings = embed_documents(documents)
        save_embeddings(embeddings, embeddings_directory)
        faiss_index = create_faiss_index(embeddings)
        save_faiss_index(faiss_index, index_filepath)
    else:
        documents = process_documents(pdf_directory)  # We still need to process documents for text retrieval
        embeddings = load_embeddings(embeddings_directory)
        faiss_index = load_faiss_index(index_filepath)

    # Embed the query
    query_embedding = embed_documents([{'text': query}])[0]

    # Find similar documents
    similar_doc_indices = search_similar_documents(faiss_index, query_embedding)

    # Prepare context
    context = "\n".join([documents[i]['text'] for i in similar_doc_indices])

    # Query Claude
    response = query_claude(query, context)

    return response

# Usage
pdf_directory = "path/to/your/pdfs"
embeddings_directory = "path/to/local/embeddings"
index_filepath = "path/to/local/faiss_index.bin"
query = "What is the main topic discussed in these documents?"
result = rag_pipeline(query, pdf_directory, embeddings_directory, index_filepath)
print(result)

In [ ]:
#User Interface:Create a simple Flask web application for user interaction
from flask import Flask, render_template, request
   from rag_orchestrator import rag_pipeline

   app = Flask(__name__)

   @app.route('/', methods=['GET', 'POST'])
   def index():
       if request.method == 'POST':
           query = request.form['query']
           pdf_directory = "path/to/your/pdfs"
           result = rag_pipeline(query, pdf_directory)
           return render_template('index.html', result=result)
       return render_template('index.html')

   if __name__ == '__main__':
       app.run(debug=True)